In [1]:
from surprise import KNNBasic
from surprise import Dataset
from surprise.model_selection import cross_validate
from collections import defaultdict
from surprise import get_dataset_dir
import io

In [2]:
data = Dataset.load_builtin('ml-100k')
trainset = data.build_full_trainset()
algo = KNNBasic()
algo.fit(trainset)

testset = trainset.build_anti_testset()

predictions = algo.test(testset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [3]:
scores = cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9844  0.9769  0.9821  0.9752  0.9790  0.9795  0.0034  
Fit time          0.32    0.31    0.34    0.31    0.38    0.33    0.03    
Test time         2.87    3.00    2.93    2.88    3.05    2.95    0.07    


In [4]:


def get_top_n(predictions, n=10):
    

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n


top_n = get_top_n(predictions, 5)


In [ ]:
def read_item_names():
    file_name = get_dataset_dir() + '/ml-100k/ml-100k/u.item'
    rid_to_name = {}
    with io.open(file_name, 'r', encoding='ISO-8859-1') as f:
        for line in f:
            line = line.split('|')
            rid_to_name[line[0]] = line[1]

    return rid_to_name

rid_to_name= read_item_names()

user_id = int(raw_input())

user = algo.trainset.to_inner_uid(unicode(user_id))


for i in top_n.get(unicode(user)):
    print rid_to_name[i[0]]

In [ ]:
user_id_neighbors = algo.get_neighbors(user, k=3)

def read_user_info():
    file_name = get_dataset_dir() + '/ml-100k/ml-100k/u.user'
    rid_to_name = {}
    with io.open(file_name, 'r', encoding='ISO-8859-1') as f:
        for line in f:
            line = line.split('|')
            rid_to_name[line[0]] = (line[1],line[2],line[3])

    return rid_to_name

rid_to_name = read_user_info()
print 'The user is','id:',user_id,'Age:',rid_to_name[unicode(user_id)][0],'Gender:',rid_to_name[unicode(user_id)][1],'Occupation:',rid_to_name[unicode(user_id)][2]

print 

print('The 3 nearest neighbors')
for i in user_id_neighbors:
    print 'id',i,'Age:',rid_to_name[unicode(i)][0],'Gender:',rid_to_name[unicode(i)][1],'Occupation:',rid_to_name[unicode(i)][2]
    
